In [ ]:
from kafka import KafkaConsumer
from datetime import datetime
from datetime import timedelta
import re

In [ ]:
consumer = KafkaConsumer(bootstrap_servers=['kafka:9092'])

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils # не хочет
import json

sc = SparkContext(appName="PythonSparkStreamingKafka_RM_01")
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, 60)
kafkaStream = KafkaUtils.createStream(ssc, 'kafka:9092', 'spark-streaming', {'last_stream':1})
parsed = kafkaStream.map(lambda v: json.loads(v[1]))

ssc.start()
ssc.awaitTermination()

In [ ]:
# функция парсинга времени и рассчета дельты
def time_delta(first_mes, second_mes):
    p = re.compile("\d{4}-\d{2}-\d{2}\w{1}\d{2}:\d{2}:\d{2}.\d{3}")
    datetime1 = datetime.strptime(p.findall(first_mes["eventTime"])[0], '%Y-%m-%dT%H:%M:%S.%f')
    datetime2 = datetime.strptime(p.findall(second_mes["eventTime"])[0], '%Y-%m-%dT%H:%M:%S.%f')
    return(round((datetime2 - datetime1).total_seconds() * 1000))

In [ ]:
# алгоритм обработки потока по 1 сообщению и создания словаря с массивами сообщений по id клиента
base_by_id = {}
cnt = 0
for message in consumer:
    print(message)
    if message["id"] not in base_by_id:
        base_by_id[message["id"]] = [message]
    elif message["id"] in base_by_id:
        base_by_id[message["id"]].append(message)
    cnt += 1
    if cnt == 20:
        break

In [ ]:
# алгоритм отбора событий по условиям задачи
first_message = {}
second_message = {}

for key, value in base_by_id.items():
    first_message = value[0]
    counter = 0
    for message in value:
#         print(message)
        if first_message["client"] == "mobile":
            first_message["chanel"] = "push"
            first_message.pop("client")
            base_by_id[key] = first_message
            print(base_by_id[key])
            break
        elif first_message["client"] == "web":
            if len(value) > 1:
                counter += 1
                second_message = value[counter]
                if (second_message["client"] == "mobile") and (time_delta(first_message, second_message) <= 300000):
                    second_message["chanel"] = "push"
                    second_message.pop("client")
                    base_by_id[key] = second_message
                    print(base_by_id[key])
                    break
                else:
                    second_message["chanel"] = "sms"
                    second_message.pop("client")
                    base_by_id[key] = second_message
                    print(base_by_id[key])
                    break
            else:
                first_message["chanel"] = "sms"
                first_message.pop("client")
                base_by_id[key] = first_message
                print(base_by_id[key])
                break

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark import SparkContext

In [ ]:
# достаем нужные значения из датасета
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Session')\
        .getOrCreate()

agg_usage = spark.read \
    .format('parquet') \
    .load("../data/agg_usage.parquet")

agg_usage.printSchema()
agg_usage.head()

In [ ]:
import json
import time
from kafka import KafkaProducer


def serializer(message):
    # message - JSON object
    return json.dumps(message).encode('utf-8')


def send_messages(producer, topic, path):
    # producer - a Kafka Producer object
    # topic - name of the Kafka Topic
    # path - path to the JSON file

    with open(path) as f:
        for jsonObj in f:
            message = json.loads(jsonObj)
            producer.send(topic, message)

    producer.flush()


def on_send_success(record_metadata):
    # record_metadata - Message sent to Kafka metadata
    print(f"Topic: {record_metadata.topic} / Partition: {record_metadata.partition} / Offset: {record_metadata.offset}")


# Create Producer
producer = KafkaProducer(bootstrap_servers=['kafka:9092'],
                         api_version=(0, 10), value_serializer=serializer)

# Send messages to particular topic
send_messages(producer, 'out_cm', base_by_id)
